# Diffusion Model for Open Dialogue

Simplified version

References
- DiffuSeq (cited below)

@inproceedings{gong2022diffuseq,
  author = {Gong, Shansan and Li, Mukai and Feng, Jiangtao and Wu, Zhiyong and Kong, Lingpeng},
  booktitle = {International Conference on Learning Representations, ICLR},
  title = {{DiffuSeq}: Sequence to Sequence Text Generation with Diffusion Models},
  year = 2023
}

@article{gong2023diffuseqv2,
  title={DiffuSeq-v2: Bridging Discrete and Continuous Text Spaces for Accelerated Seq2Seq Diffusion Models},
  author={Gong, Shansan and Li, Mukai and Feng, Jiangtao and Wu, Zhiyong and Kong, Lingpeng},
  journal={arXiv preprint arXiv:2310.05793},
  year={2023}
}


## Dataset

- Use Commonsense Conversation dataset (from Reddit)


in diffuseq text_datasets.py some steps to load the dataset itself

- [ ] prepare datasets for training and validation in the format (stored as jsonl file?)
```
{"src": "", "train": ""}
```

- word embeddings (to be loaded?)
- use a corpus


## Training

Note that, in DiffuSeq, a model file is created to store all training progress, configuration etc. (in bash format poitning to raw files?)

- denoise rate ?
- using updates in v2 diffuseq took it from 2 days -> 11 hr learning time

Load the tokenizer

For simplicity choose BERT rather than custom

In [2]:
# Load tokenizer from bert
# TODO check if this is the best tokenizer for Commonsense Conversation dataset
import torch
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# sep_token_id = tokenizer.sep_token_id
# pad_token_id = tokenizer.pad_token_id


Diffusion-LM (Li et al., 2022):

 embedding function $EMB(w)$ to map the discrete text $w$ into a continuous space.

load the model embeddings
```
model_weight, tokenizer = load_model_emb(args, tokenizer)
```

Note the implementation details in DiffuSeq (first version) is
"The maximum sequence length is 128, with embedding dimension d = 128, diffusion steps T = 2000
and a square-root noise schedule."

How is it different in v2 or other papers?

In [ ]:
# choose hidden_dim

# choose embedding dimension
d=128 
# TODO choose embedding dimension suitable to the task

model = torch.nn.Embedding(tokenizer.vocab_size, 128)

# initialize random embeddings
torch.nn.init.normal_(model.weight)
torch.save(model.state_dict(), path_save)
# os.sync() FIXME required?

# FIXME need to implement saving and reloading?


In [ ]:
    ### random emb or pre-defined embedding like glove embedding. You can custome your own init here.


load the text data from file

In [ ]:
# read in the data in training data json file 
# TODO do this in a different way 
# FIXME what is the .strip() used for specifically?

import json

data_dir = "./datasets/sample"
path = f'{data_dir}/train.jsonl'

sentence_lst = {'src':[], 'trg': []}
with open(path, 'r') as f_reader:
        for row in f_reader:
            content = json.loads(row)
            sentence_lst['src'].append(content['src'].strip())
            sentence_lst['trg'].append(content['trg'].strip())

create the model and diffusion
```
model, diffusion = create_model_and_diffusion(
        **args_to_dict(args, load_defaults_config().keys())
````

create schedule sampler? is it required?

run the training loop according to batch size, etc.